In [1]:
import awkward1 as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import numpy as np
from tqdm import tqdm,trange
import matplotlib.pyplot as plt
import numba

In [2]:
#infile = "data/signal/wza_UL18.root" # --signal
infile='data/data/7B56E217-555E-1C41-9494-491849A9835F_skim_2ElIdPt20.root' # --data 
#infile = 'data/data/CA42F3A2-614F-4A4F-AF18-F6E66CDA2C85_skim_2ElIdPt20.root'
#infile = 'data/Ntuple/EGamma_Run2018B.root'
#infile="data/mc/59AB328B-F0E3-F544-98BB-E5E55577C649_skim_2ElIdPt20.root" # --mc


year='2018'

In [3]:
events = NanoEventsFactory.from_root(infile, schemaclass=NanoAODSchema).events()

In [4]:
# Trigger set
doubleelectron_triggers  ={
    '2018': [
            "Ele23_Ele12_CaloIdL_TrackIdL_IsoVL", # Recomended
            ]
}



singleelectron_triggers = { #2017 and 2018 from monojet, applying dedicated trigger weights
        '2016': [
            'Ele27_WPTight_Gsf',
            'Ele105_CaloIdVT_GsfTrkIdT'
        ],
        '2017': [
            'Ele35_WPTight_Gsf',
            'Ele115_CaloIdVT_GsfTrkIdT',
            'Photon200'
        ],
        '2018': [
            'Ele32_WPTight_Gsf',    # Recomended
        ]
    }

In [5]:
# << flat dim helper function >>
def flat_dim(arr):

#   if len(ak.to_numpy(arr).shape) > 1:
#       sub_arr = ak.flatten(arr)
#   else:
#       sub_arr = arr
    sub_arr = ak.flatten(arr)
    mask = ~ak.is_none(sub_arr)

    return ak.to_numpy(sub_arr[mask])
# << drop na helper function >>
def drop_na(arr):

    mask = ~ak.is_none(arr)

    return arr[mask]
# << drop na helper function >>
def drop_na_np(arr):

    mask = ~np.isnan(arr)

    return arr[mask]
# << Sort by PT  helper function >>
def sort_by_pt(ele,pho,jet):
    ele = ele[ak.argsort(ele.pt,ascending=False,axis=1)]
    pho = pho[ak.argsort(pho.pt,ascending=False,axis=1)]
    jet = jet[ak.argsort(jet.pt,ascending=False,axis=1)]

    return ele,pho,jet

In [6]:
# double lepton trigger
is_double_ele_trigger=True
if not is_double_ele_trigger:
    double_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    double_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in doubleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        double_ele_triggers_arr = double_ele_triggers_arr | events.HLT[path]


# single lepton trigger
is_single_ele_trigger=True
if not is_single_ele_trigger:
    single_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    single_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in singleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        single_ele_triggers_arr = single_ele_triggers_arr | events.HLT[path]

# Sort particle order by PT  # RunD --> has problem
events.Electron,events.Photon,events.Jet = sort_by_pt(events.Electron,events.Photon,events.Jet)

Initial_events = events
#events = events[single_ele_triggers_arr | double_ele_triggers_arr]
events = events[double_ele_triggers_arr]

cut1 = np.ones(len(events))
# Particle Identification
Electron = events.Electron
Photon = events.Photon
MET = events.MET


In [7]:
print("#### 0. Trigger: {0} -> {1}".format(len(Initial_events), len(events)))

#### 0. Trigger: 58612 -> 50077


### 1. Channel selection with basic cut applied (without ID)

In [8]:
def Particle_selection(ele,pho):


    EleSelmask = EleSelmask = ((ele.pt > 10) & (np.abs(ele.eta + ele.deltaEtaSC) < 1.4442) & (ele.cutBased > 2) & (abs(ele.dxy) < 0.05) & (abs(ele.dz) < 0.1)) | \
        ((ele.pt > 10) & (np.abs(ele.eta + ele.deltaEtaSC) > 1.5660) & (np.abs(ele.eta + ele.deltaEtaSC) < 2.5) & (ele.cutBased > 2) & (abs(ele.dxy) < 0.1) & (abs(ele.dz) < 0.2))


    # Photon selection
    isgap_mask = (abs(pho.eta) < 1.442)  |  ((abs(pho.eta) > 1.566) & (abs(pho.eta) < 2.5))
    Pixel_seed_mask = ~pho.pixelSeed
    PhoSelmask = (pho.pt > 20)  & isgap_mask &  Pixel_seed_mask


    return EleSelmask,PhoSelmask

In [9]:
Electron_mask, Photon_mask  = Particle_selection(Electron,Photon)
Ele_channel_mask = ak.num(Electron[Electron_mask])  > 1
Pho_channel_mask = ak.num(Photon[Photon_mask]) > 0
Ele_channel_events = events[Ele_channel_mask & Pho_channel_mask]

In [10]:
print("#### 1. Channel selection with basic cut applied {0}->{1} {2}%".format(len(events),len(Ele_channel_events), len(Ele_channel_events)/len(events) * 100))

#### 1. Channel selection with basic cut applied 50077->1608 3.21105497533798%


In [11]:
# Particle array
Electron = Ele_channel_events.Electron
Photon  = Ele_channel_events.Photon
Jet     = Ele_channel_events.Jet
MET      = Ele_channel_events.MET

Electron_mask,Photon_mask = Particle_selection(Electron,Photon)
Electron = Electron[Electron_mask]
Photon  = Photon[Photon_mask]

### 2. Select OSSF electron pairs

In [12]:
##-----------  Cut flow3: Electron Selection --> OSSF 
# OSSF index maker
@numba.njit
def find_2lep(events_leptons,builder):
    for leptons in events_leptons:

        builder.begin_list()
        nlep = len(leptons)
        for i0 in range(nlep):
            for i1 in range(i0+1,nlep):
                if leptons[i0].charge + leptons[i1].charge != 0: continue;
                
                if nlep == 2:
                    builder.begin_tuple(2)
                    builder.index(0).integer(i0)
                    builder.index(1).integer(i1)
                    builder.end_tuple()  

            
                else:
                    for i2 in range(nlep):
                        if len({i0,i1,i2}) < 3: continue;
                        builder.begin_tuple(3)
                        builder.index(0).integer(i0)
                        builder.index(1).integer(i1)
                        builder.index(2).integer(i2)
                        builder.end_tuple()
        builder.end_list()
    return builder

In [13]:
ossf_idx = find_2lep(Electron,ak.ArrayBuilder()).snapshot()

ossf_idx

<Array [[(0, 1)], [(0, 1), ... 1)], [(0, 1)]] type='1608 * var * (int64, int64)'>

In [14]:
# OSSF cut
ossf_mask = ak.num(ossf_idx) >= 1
ossf_idx = ossf_idx[ossf_mask]

Ele_channel_events = Ele_channel_events[ossf_mask]
Electron= Electron[ossf_mask]
Photon= Photon[ossf_mask]
Jet= Jet[ossf_mask]
MET = MET[ossf_mask]

In [15]:
print("#### 2. OSSF {0}".format(len(Ele_channel_events)))

#### 2. OSSF 1584


In [16]:
Double_electron = [Electron[ossf_idx[idx]] for idx in "01"]

In [17]:
from coffea.nanoevents.methods import vector
ak.behavior.update(vector.behavior)

def TLorentz_vector(vec):
    vec = ak.zip(
    {
                "x":vec.x,
                "y":vec.y,
                "z":vec.z,
                "t":vec.t
    },
    with_name = "LorentzVector"
    )
    return vec

def TLorentz_vector_cylinder(vec):

    vec = ak.zip(
    {
         "pt": vec.pt,
         "eta": vec.eta,
         "phi": vec.phi,
         "mass": vec.mass,
    },
    with_name="PtEtaPhiMLorentzVector",
    )

    return vec


In [18]:
Diele    = ak.zip({"lep1":Double_electron[0],
                   "lep2":Double_electron[1],
                     "p4":TLorentz_vector(Double_electron[0]+Double_electron[1])})

In [19]:
bestZ_idx = ak.singletons(ak.argmin(abs(Diele.p4.mass - 91.1876), axis=1))
Diele = Diele[bestZ_idx]

leading_ele, subleading_ele  = ak.flatten(TLorentz_vector_cylinder(Diele.lep1)),ak.flatten(TLorentz_vector_cylinder(Diele.lep2))

### 3. Photon selection with dR

In [20]:
##-----------  Cut flow4: Photon "cleaning"  -- Basis of array-dimension
def make_DR(ele1,ele2,pho,jet):

    dR_e1pho  = ele1.delta_r(pho) # dR pho,ele1
    dR_e2pho  = ele2.delta_r(pho) # dR pho,ele2
    dR_phojet = jet[:,0].delta_r(pho) # dR pho,jet # #--> Need check

    #dR_mask    = (dR_e1pho > 0.4) & (dR_e2pho > 0.4)&  (dR_e3pho > 0.4) & (dR_phojet > 0.4) #--> Need check
    dR_mask = (dR_e1pho > 0.4) & (dR_e2pho > 0.4)

    #return dR_mask,dR_e1pho,dR_e2pho,dR_e3pho,dR_phojet #--> Need check
    return dR_mask,dR_e1pho,dR_e2pho,dR_phojet

In [21]:
dR_mask,dR_e1pho,dR_e2pho,dR_phojet  = make_DR(leading_ele,subleading_ele,Photon,Jet)
Photon = Photon[dR_mask]

In [22]:
Ele_channel_events  = Ele_channel_events[ak.num(Photon) > 0]
Diele               = Diele[ak.num(Photon) > 0]
Jet                 = Jet[ak.num(Photon) > 0]
MET                 = MET[ak.num(Photon) > 0]
Photon              = Photon[ak.num(Photon) > 0] # Beware the order! Photon must be located last!

In [23]:
print("#### 3. Photon dR {0}".format(len(Ele_channel_events)))

#### 3. Photon dR 943


### 4. Make Fake template (Inverse sieie and IsoChg) 

In [24]:
#@numba.njit ## Numba compile -- Boost!
def PhotonVID(vid, idBit):
    rBit = 0
    for x in range(0, 7):
        rBit |= (1 << x) if ((vid >> (x * 2)) & 0b11 >= idBit) else 0
    return rBit


#@numba.njit ## Numba compile -- Boost!
def make_fake_obj_mask(Pho):

    isFakeObj=[]
    for eventIdx,pho in enumerate(Pho):

        NFake=0
        if len(pho) < 1: continue;
        for phoIdx,_ in enumerate(pho):
            vid = Pho[eventIdx][phoIdx].vidNestedWPBitmap
            vid_cuts1 = PhotonVID(vid,1) # Loose photon
            vid_cuts2 = PhotonVID(vid,2) # Medium photon
            vid_cuts3 = PhotonVID(vid,3) # Tight photon

            # Field name
            # |0|0|0|0|0|0|0| 
            # |IsoPho|IsoNeu|IsoChg|Sieie|hoe|scEta|PT|

            # 1. Turn off cut (ex turn off Sieie
            # |1|1|1|0|1|1|1| = |1|1|1|0|1|1|1|

            # 2. Inverse cut (ex inverse Sieie)
            # |1|1|1|1|1|1|1| = |1|1|1|0|1|1|1|


            #if (vid_cuts2 & 0b1101111 == 0b1101111): # without Isochg
            if (vid_cuts2 & 0b1111111 == 0b1110111) & (~Pho[eventIdx][phoIdx].pixelSeed): # Inverse Sieie
            #if (vid_cuts2 & 0b1110111 == 0b1110111): # Without Sieie

                NFake +=1
                
                # --Wgamma AN option -- #
                '''
                if Pho[eventIdx][phoIdx].isScEtaEB:
                    if Pho[eventIdx][phoIdx].sieie < 0.01015 * 1.75:
                        NFake+=1

                elif Pho[eventIdx][phoIdx].isScEtaEE:
                    if Pho[eventIdx][phoIdx].sieie < 0.0272 * 1.75:
                        NFake+=1

                else: continue;
                '''

        if NFake == 1:
            isFakeObj.append(True)
        else:
            isFakeObj.append(False)

    return isFakeObj

In [25]:
#@numba.njit ## Numba compile -- Boost!
def make_IsochgSide_mask(Pho):

    isIsoSideObj=[]
    for eventIdx,pho in enumerate(Pho):

        NFake=0
        if len(pho) < 1: continue;
        for phoIdx,_ in enumerate(pho):
            vid = Pho[eventIdx][phoIdx].vidNestedWPBitmap
            vid_cuts1 = PhotonVID(vid,1) # Loose photon
            vid_cuts2 = PhotonVID(vid,2) # Medium photon
            vid_cuts3 = PhotonVID(vid,3) # Tight photon


            if (vid_cuts2 & 0b1111111 == 0b1101111): # Inverse Isochg
                isochg = Pho[eventIdx][phoIdx].pfRelIso03_chg
                
                
                # -- Wgamma AN option == #
                #if (isochg > 4) & (isochg < 10):
                NFake+=1

        if NFake == 1:
            isIsoSideObj.append(True)
        else:
            isIsoSideObj.append(False)

    return isIsoSideObj

In [26]:
isFakeObj = make_fake_obj_mask(Photon)
isIsoSideObj = make_IsochgSide_mask(Photon)

In [27]:
Ele_channel_fake_template = Ele_channel_events[np.array(isIsoSideObj) | np.array(isFakeObj)]
Diele               = Diele[np.array(isIsoSideObj) | np.array(isFakeObj)]
Jet                 = Jet[np.array(isIsoSideObj) | np.array(isFakeObj)]
MET                 = MET[np.array(isIsoSideObj) | np.array(isFakeObj)]
Photon              = Photon[np.array(isIsoSideObj) | np.array(isFakeObj)] 

In [28]:
print("#### 4. Fake template {0}".format(len(Ele_channel_fake_template )))

#### 4. Fake template 79


### 5. Event Selection

In [29]:
def make_leading_pair(target,base):
            return target[ak.argmax(base.pt,axis=1,keepdims=True)]

leading_pho     = make_leading_pair(Photon,Photon)

In [30]:
##-----------  Cut flow5 and 6: Event Selection

# bjet veto
bJet_selmask = (Jet.btagCMVA > -0.5844)
bJet_veto    = ak.num(Jet[bJet_selmask])==0
cut5 = np.ones(ak.sum(ak.num(leading_pho[bJet_veto] > 0 ))) * 5

# Z mass window
zmass_window_mask = ak.firsts((Diele.p4.mass) > 60 | (Diele.p4.mass < 120))

# M(eea) cuts 
eeg_vec           = Diele.p4 + leading_pho
Meeg_mask         = ak.firsts(eeg_vec.mass > 120)

# Mask
Event_sel_mask   = bJet_veto & zmass_window_mask & Meeg_mask

Diele_sel   = Diele[Event_sel_mask]

leading_pho_sel   = leading_pho[Event_sel_mask]
# Photon  EE and EB

isEE_mask = leading_pho.isScEtaEE
isEB_mask = leading_pho.isScEtaEB
Pho_EE = leading_pho[isEE_mask & Event_sel_mask]
Pho_EB = leading_pho[isEB_mask & Event_sel_mask]

MET_sel           = MET[Event_sel_mask]

In [31]:
print("#### 5. Event selection: {0}".format(len(Diele_sel)))

#### 5. Event selection: 33


In [34]:
Pho_EE_pt     = flat_dim(Pho_EE.pt)
Pho_EE_eta    = flat_dim(Pho_EE.eta)
Pho_EE_phi    = flat_dim(Pho_EE.phi)
Pho_EE_Isochg = Pho_EE.pfRelIso03_chg
Pho_EE_Sieie  = Pho_EE.sieie


Pho_EB_pt  = flat_dim(Pho_EB.pt)
Pho_EB_eta = flat_dim(Pho_EB.eta)
Pho_EB_phi = flat_dim(Pho_EB.phi)
Pho_EE_Isochg = Pho_EE.pfRelIso03_chg
Pho_EE_Sieie  = Pho_EE.sieie



<Array [[], [], [], [], ... [], [], [], []] type='79 * option[var * ?float32[par...'>